<a href="https://colab.research.google.com/github/singhsa3/CharityML/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import numpy as np

In [132]:
import torch.nn as nn
import torch.nn.functional as F


class Fp32GroupNorm(nn.GroupNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, input):
        output = F.group_norm(
            input.float(),
            self.num_groups,
            self.weight.float() if self.weight is not None else None,
            self.bias.float() if self.bias is not None else None,
            self.eps,
        )
        return output.type_as(input)

In [91]:
from google.colab import drive
drive.mount('/content/drive')
pathG='/content/drive/MyDrive/Pract/data'
cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
import pickle
with open(pathG+"/featuresdict.pickle", "rb") as f:
  features=pickle.load(f)

In [93]:
with open(pathG+"/filenameslist.pickle", "rb") as f:
  fnames=pickle.load(f)

In [94]:
import pandas as pd
df=pd.read_csv(pathG+"/labels/Yared Alemu_fear.csv")
df = df[(df['name'].isin (features.keys()))].reset_index()
df.head()

,index,Unnamed: 0,therapist,name,emotion_type,rating,emotion
0,0,8015,Yared Alemu,6708_53113_1589932800.wav,fear,high,1
1,1,5925,Yared Alemu,7163_53113_1603843200.wav,fear,high,1
2,3,4565,Yared Alemu,52757_53113_1569456000.wav,fear,medium,0
3,4,4530,Yared Alemu,52757_53113_1574294400.wav,fear,low,0
4,5,6625,Yared Alemu,1940_39117_1592956800.wav,fear,low,0


In [95]:
x=features[df.iloc[0][3]]
x= np.pad(x, ((0,0),(10, 7000-x.shape[1])), 'constant')
arr =np.expand_dims(x, axis=0)
#print(arr.shape)
for i,row in df.iterrows():
  if i>0:
    ft = features[row['name']]
    ft= np.pad(ft, ((0,0),(10, 7000-ft.shape[1])), 'constant')
    arr2 = np.expand_dims(ft, axis=0)
    #print(arr2.shape)
    arr = np.vstack((arr,arr2))
labels=np.array(df.emotion)

In [96]:
arr.shape


(500, 128, 7010)

In [136]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(arr, labels, test_size = 0.3)

In [137]:
# converting training images into torch format
#train_x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1], train_x.shape[2])
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape
# converting validation images into torch format
#val_x = val_x.reshape(val_x.shape[0], 1, val_x.shape[1], val_x.shape[2])
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([150, 128, 7010]), torch.Size([150]))

In [138]:
import torch
torch.cuda.empty_cache()
!nvidia-smi
a = torch.cuda.memory_allocated(0)
c = torch.cuda.memory_reserved(0)
print(a,c)

Fri Jun  3 03:38:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |  13005MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [139]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [101]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(224256, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [164]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers =  Sequential(
        Conv1d(128, 512, kernel_size=(10,), stride=(5,), bias=False),
        Dropout(p=0.0, inplace=False),
        Fp32GroupNorm(512, 512, eps=1e-05, affine=True),
        GELU(),
     
        Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU(),
      
        Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU(),
       
        Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU(),
 
   
        Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU(),
      

        Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU(),
    
   
        Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False),
        Dropout(p=0.0, inplace=False),
        GELU()
      )

        self.linear_layers = Sequential(
            Linear(10752, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

     
    
  

In [162]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv1d(128, 512, kernel_size=(10,), stride=(5,), bias=False)
    (1): Dropout(p=0.0, inplace=False)
    (2): Fp32GroupNorm(128, 512, eps=1e-05, affine=True)
    (3): GELU()
    (4): Conv1d(128, 512, kernel_size=(3,), stride=(2,), bias=False)
    (5): Dropout(p=0.0, inplace=False)
    (6): GELU()
    (7): Conv1d(128, 512, kernel_size=(3,), stride=(2,), bias=False)
    (8): Dropout(p=0.0, inplace=False)
    (9): GELU()
    (10): Conv1d(128, 512, kernel_size=(3,), stride=(2,), bias=False)
    (11): Dropout(p=0.0, inplace=False)
    (12): GELU()
    (13): Conv1d(128, 512, kernel_size=(3,), stride=(2,), bias=False)
    (14): Dropout(p=0.0, inplace=False)
    (15): GELU()
    (16): Conv1d(128, 512, kernel_size=(2,), stride=(2,), bias=False)
    (17): Dropout(p=0.0, inplace=False)
    (18): GELU()
    (19): Conv1d(128, 512, kernel_size=(2,), stride=(2,), bias=False)
    (20): Dropout(p=0.0, inplace=False)
    (21): GELU()
  )
  (linear_layers): Sequen

In [103]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [163]:
# Define the batch size and the number of epochs
BATCH_SIZE = 10
N_EPOCHS = 100
torch.cuda.empty_cache()
# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)
# empty list to store training losses
train_losses = [100,100,100,100,100]
# empty list to store validation losses
val_losses = []

# Loop over epochs
for epoch in range(N_EPOCHS):
    mnl = np.mean(train_losses)
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        #x_val = val_x.cuda()
        #y_val = val_y.cuda()

        y_batch_pred = model(x_batch)
        #output_val = model(x_val)

        loss = criterion(y_batch_pred, y_batch)
        #loss_val = criterion(output_val, y_val)
        #train_losses.append(loss)
        #val_losses.append(loss_val)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Every 100 batches, print the loss for this batch
        # as well as the number of examples processed so far 
        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")    
    train_losses.pop()
    train_losses.insert(0,loss.detach().cpu().numpy())
    #if np.mean(train_losses)>mnl*1.10:
      #break
torch.cuda.empty_cache()

Epoch 1
-------------------------------


RuntimeError: ignored

In [157]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

0.5171428571428571

In [156]:
from sklearn.metrics import accuracy_score
# prediction for validation set
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)

0.5